In [75]:
%load_ext autoreload
%autoreload 2
%matplotlib qt
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
from glob import glob
#set path for my_tools script
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [79]:
#list all relevant files
filez = sorted(glob('/media/onno/Algemeen/Thesis/GFS_mean/*phasespeed*.nc'))
#Put all lead day errors into one dataset
datasets = []
for i, file in enumerate(filez):
    ds_single = xr.open_dataset(file,decode_times=False)
    datasets.append(ds_single)
ds = xr.concat(datasets,dim='lead').squeeze()

In [80]:
#Only select values between 20 and 80 degrees north
ds =ds.sel(lat=slice(80,20))
#Seperating the data in bins of 10 degrees
latz = [(20,30),(30,40),(40,50),(50,60),(60,70),(70,80)]
leads = np.linspace(0,240,11)
mean_lat_lead = []
for lat in latz:
    mean_lat_each_lead=[]
    for lead in leads:   
        #Calculating the mean compensating for cosine altitude weighting
        ds_lat = ds.sel(lat=slice(lat[1],lat[0]),lead=lead)
        ds_lat_mean = my_tools.weighted_average_area(ds_lat)
        mean_lat_each_lead.append(float(ds_lat_mean))
    mean_lat_lead.append(mean_lat_each_lead)
mean_lat_lead = np.array(mean_lat_lead)
print(mean_lat_lead)        
    

[[ 0.         -0.22269797 -0.17327106 -0.20218268 -0.22369966 -0.18509324
  -0.15430865 -0.21805076 -0.2650072  -0.2757006  -0.20581493]
 [ 0.         -0.31432471 -0.38909486 -0.41941345 -0.43612447 -0.43406713
  -0.42286304 -0.39944455 -0.47697347 -0.45164818 -0.45557433]
 [ 0.         -0.31115746 -0.36487457 -0.42048326 -0.43812075 -0.44898364
  -0.44459113 -0.42003953 -0.4579629  -0.49149889 -0.51593596]
 [ 0.         -0.28970933 -0.28760779 -0.31008792 -0.32072955 -0.3322449
  -0.34915251 -0.33285066 -0.31908256 -0.35032564 -0.35045654]
 [ 0.         -0.204695   -0.20229112 -0.24367282 -0.23637258 -0.24479944
  -0.2333764  -0.21464933 -0.1780483  -0.18775339 -0.17854762]
 [ 0.         -0.14047706 -0.14890774 -0.15519413 -0.16265881 -0.15750428
  -0.16212061 -0.18508458 -0.1609142  -0.17138647 -0.17704977]]


In [82]:
#Creating line graph
legendz = ['20-30 °N','30-40 °N','40-50 °N','50-60 °N','60-70 °N','70-80 °N']
colorz = ['#930717','#AF0F19','#CF4A0C','#E1A023','#EBD110','#6A9D16']
fig,ax = plt.subplots()

for i in range(len(mean_lat_lead)):
    ax.plot(mean_lat_lead[i],label=legendz[i],color=colorz[i])
    ax.legend(loc='upper right')
    ax.set_xlabel("Lead time (days)")
    ax.set_ylabel('Mean error in Cp (m/s)')
    ax.set_ylim(-0.6,0)
    ax.set_xticks(np.arange(11))
    ax.grid(b=True,which='major',axis='y')
    ax.margins(0)
fig.suptitle('GFS Mean Error in Phase Speed for each Latitude Group compared to ERA5')
fig.show()